In [11]:
# Setup: Download Codebase and Verify Environment
import os
import subprocess
import sys
from pathlib import Path

# Repository information
REPO_URL = "https://github.com/yimianxyz/homepage.git"
BRANCH = "neuro-predator"
REPO_DIR = "homepage"

def download_codebase():
    """Download the codebase from GitHub if not already present"""
    if os.path.exists(REPO_DIR):
        print(f"Repository directory '{REPO_DIR}' already exists.")
        try:
            os.chdir(REPO_DIR)
            # Check current branch
            result = subprocess.run(['git', 'branch', '--show-current'],
                                  capture_output=True, text=True, check=True)
            current_branch = result.stdout.strip()
            if current_branch != BRANCH:
                print(f"Switching to branch '{BRANCH}'...")
                subprocess.run(['git', 'checkout', BRANCH], check=True)
            # Pull latest changes
            print("Updating repository...")
            subprocess.run(['git', 'pull', 'origin', BRANCH], check=True)
            print(f"✅ Repository updated successfully!")
        except subprocess.CalledProcessError as e:
            print(f"❌ Error updating repository: {e}")
    else:
        print(f"Cloning repository from {REPO_URL} (branch: {BRANCH})...")
        try:
            subprocess.run(['git', 'clone', '-b', BRANCH, REPO_URL, REPO_DIR], check=True)
            print(f"✅ Repository cloned successfully!")
            os.chdir(REPO_DIR)
        except subprocess.CalledProcessError as e:
            print(f"❌ Error cloning repository: {e}")
            return False

    # Verify key files exist
    key_files = [
        'config/constants.py',
        'simulation/processors/input_processor.py',
        'simulation/state_manager/state_manager.py',
        'policy/human_prior/closest_pursuit_policy.py',
        'export_to_js.py'
    ]

    missing_files = []
    for file_path in key_files:
        if not os.path.exists(file_path):
            missing_files.append(file_path)

    if missing_files:
        print(f"⚠️  Warning: Some key files are missing:")
        for file_path in missing_files:
            print(f"  - {file_path}")
        return False

    print(f"✅ All key files found!")
    print(f"📁 Working directory: {os.getcwd()}")
    return True

# Download and setup codebase
success = download_codebase()

if success:
    print("\n🎉 Setup complete! Ready for RL training.")
else:
    print("❌ Setup failed. Please check the errors above and try again.")
    raise RuntimeError("Failed to setup codebase")


Cloning repository from https://github.com/yimianxyz/homepage.git (branch: neuro-predator)...
✅ Repository cloned successfully!
✅ All key files found!
📁 Working directory: /content/homepage/homepage/homepage

🎉 Setup complete! Ready for RL training.


In [12]:
# Import Dependencies and Setup Configuration
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import random
import json
import matplotlib.pyplot as plt
from datetime import datetime
import time
from collections import deque, defaultdict
from typing import Dict, List, Any, Tuple, Optional
import copy

# Ensure we're in the correct directory and add to Python path
project_root = Path.cwd()
if project_root.name != 'homepage':
    print(f"⚠️  Warning: Current directory is '{project_root.name}', expected 'homepage'")
    print("Make sure the first cell downloaded the repository correctly.")

if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

# Import project modules
try:
    from config.constants import CONSTANTS
    from simulation.processors import InputProcessor, ActionProcessor
    from simulation.state_manager import StateManager
    from simulation.random_state_generator import RandomStateGenerator
    from policy.human_prior.closest_pursuit_policy import create_closest_pursuit_policy

    print(f"✅ Successfully imported all simulation modules")
    print(f"📁 Project root: {project_root}")
    print(f"🔧 Key constants: MAX_DISTANCE={CONSTANTS.MAX_DISTANCE}, BOID_MAX_SPEED={CONSTANTS.BOID_MAX_SPEED}")

except ImportError as e:
    print(f"❌ Failed to import modules: {e}")
    print("Make sure the repository was downloaded correctly in the first cell.")
    raise

# RL Training Configuration
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🚀 Device: {DEVICE}")

# Environment Configuration
ENV_CONFIG = {
    'max_episode_steps': 3000,         # Prevent endless episodes
    'reward_window': 50,               # Last N steps before catch get reward
    'base_catch_reward': 1.0,          # Base reward for catching a boid
    'canvas_width_range': (320, 3840), # Same as SL data
    'canvas_height_range': (320, 2160), # Same as SL data
    'boid_count_range': (1, 50),       # Same as SL data
}

# PPO Configuration
PPO_CONFIG = {
    'learning_rate': 3e-5,        # Lower than SL training
    'batch_size': 256,            # Same as SL
    'rollout_length': 2048,       # Steps per update
    'epochs_per_update': 4,       # PPO epochs
    'clip_epsilon': 0.1,          # PPO clip parameter
    'value_loss_coef': 0.5,       # Value loss weight
    'entropy_coef': 0.01,         # Exploration bonus
    'gae_lambda': 0.95,           # GAE parameter
    'gamma': 0.99,                # Discount factor
}

print(f"\n📋 Configuration:")
print(f"  Environment: {ENV_CONFIG}")
print(f"  PPO: {PPO_CONFIG}")
print(f"✅ Configuration loaded successfully!")


✅ Successfully imported all simulation modules
📁 Project root: /content/homepage/homepage/homepage
🔧 Key constants: MAX_DISTANCE=2000, BOID_MAX_SPEED=3.5
🚀 Device: cuda

📋 Configuration:
  Environment: {'max_episode_steps': 3000, 'reward_window': 50, 'base_catch_reward': 1.0, 'canvas_width_range': (320, 3840), 'canvas_height_range': (320, 2160), 'boid_count_range': (1, 50)}
  PPO: {'learning_rate': 3e-05, 'batch_size': 256, 'rollout_length': 2048, 'epochs_per_update': 4, 'clip_epsilon': 0.1, 'value_loss_coef': 0.5, 'entropy_coef': 0.01, 'gae_lambda': 0.95, 'gamma': 0.99}
✅ Configuration loaded successfully!


In [13]:
# Load SL Trained Model as Starting Point
# First, we need the transformer model definition from SL training

class GEGLU(nn.Module):
    def forward(self, x):
        x, gate = x.chunk(2, dim=-1)
        return x * torch.nn.functional.gelu(gate)

class TransformerLayer(nn.Module):
    def __init__(self, d_model, n_heads, ffn_hidden, dropout=0.1):
        super().__init__()
        self.d_model = d_model
        self.n_heads = n_heads

        self.norm1 = nn.LayerNorm(d_model)
        self.self_attn = nn.MultiheadAttention(d_model, n_heads, dropout=dropout, batch_first=True)

        self.norm2 = nn.LayerNorm(d_model)

        # GEGLU FFN with separate projections for export compatibility
        self.ffn_gate_proj = nn.Linear(d_model, ffn_hidden)
        self.ffn_up_proj = nn.Linear(d_model, ffn_hidden)
        self.ffn_down_proj = nn.Linear(ffn_hidden, d_model)

    def forward(self, x, padding_mask=None):
        # Self-attention with residual
        normed = self.norm1(x)
        attn_out, _ = self.self_attn(normed, normed, normed, key_padding_mask=padding_mask)
        x = x + attn_out

        # FFN with residual
        normed = self.norm2(x)
        gate = torch.nn.functional.gelu(self.ffn_gate_proj(normed))
        up = self.ffn_up_proj(normed)
        ffn_out = self.ffn_down_proj(gate * up)
        x = x + ffn_out

        return x

class TransformerPredictor(nn.Module):
    def __init__(self, d_model=128, n_heads=8, n_layers=4, ffn_hidden=512, max_boids=50, dropout=0.1):
        super().__init__()
        self.d_model = d_model
        self.n_heads = n_heads
        self.n_layers = n_layers
        self.ffn_hidden = ffn_hidden
        self.max_boids = max_boids

        # CLS token embedding
        self.cls_embedding = nn.Parameter(torch.randn(d_model))

        # Type embeddings
        self.type_embeddings = nn.ParameterDict({
            'cls': nn.Parameter(torch.randn(d_model)),
            'ctx': nn.Parameter(torch.randn(d_model)),
            'predator': nn.Parameter(torch.randn(d_model)),
            'boid': nn.Parameter(torch.randn(d_model))
        })

        # Input projections
        self.ctx_projection = nn.Linear(2, d_model)  # canvas_width, canvas_height
        self.predator_projection = nn.Linear(4, d_model)  # velX, velY, 0, 0 (padded to 4D)
        self.boid_projection = nn.Linear(4, d_model)  # relX, relY, velX, velY

        # Transformer layers
        self.transformer_layers = nn.ModuleList([
            TransformerLayer(d_model, n_heads, ffn_hidden, dropout)
            for _ in range(n_layers)
        ])

        # Output projection
        self.output_projection = nn.Linear(d_model, 2)  # predator action [x, y]

    def forward(self, structured_inputs, padding_mask=None):
        batch_size = len(structured_inputs) if isinstance(structured_inputs, list) else 1

        # Handle single sample vs batch
        if isinstance(structured_inputs, dict):
            structured_inputs = [structured_inputs]
            batch_size = 1

        # Build token sequences for each sample in batch
        sequences = []
        masks = []

        for sample in structured_inputs:
            tokens = []

            # CLS token
            cls_token = self.cls_embedding + self.type_embeddings['cls']
            tokens.append(cls_token)

            # Context token
            ctx_input = torch.tensor([sample['context']['canvasWidth'], sample['context']['canvasHeight']],
                                   dtype=torch.float32, device=self.cls_embedding.device)
            ctx_token = self.ctx_projection(ctx_input) + self.type_embeddings['ctx']
            tokens.append(ctx_token)

            # Predator token - expand to 4D
            predator_input = torch.tensor([sample['predator']['velX'], sample['predator']['velY'], 0.0, 0.0],
                                        dtype=torch.float32, device=self.cls_embedding.device)
            predator_token = self.predator_projection(predator_input) + self.type_embeddings['predator']
            tokens.append(predator_token)

            # Boid tokens
            sample_mask = [False, False, False]  # CLS, CTX, Predator are not padding

            for boid in sample['boids']:
                boid_input = torch.tensor([boid['relX'], boid['relY'], boid['velX'], boid['velY']],
                                        dtype=torch.float32, device=self.cls_embedding.device)
                boid_token = self.boid_projection(boid_input) + self.type_embeddings['boid']
                tokens.append(boid_token)
                sample_mask.append(False)

            # Pad to max_boids + 3 (CLS + CTX + Predator)
            while len(tokens) < self.max_boids + 3:
                padding_token = torch.zeros(self.d_model, device=self.cls_embedding.device)
                tokens.append(padding_token)
                sample_mask.append(True)  # Mark as padding

            sequences.append(torch.stack(tokens))
            masks.append(sample_mask)

        # Stack sequences
        x = torch.stack(sequences)  # [batch_size, seq_len, d_model]

        # Create padding mask
        if padding_mask is None:
            padding_mask = torch.tensor(masks, dtype=torch.bool, device=x.device)

        # Pass through transformer layers
        for layer in self.transformer_layers:
            x = layer(x, padding_mask)

        # Extract CLS token and project to output
        cls_output = x[:, 0]  # [batch_size, d_model]
        action = self.output_projection(cls_output)  # [batch_size, 2]

        # Apply tanh to ensure [-1, 1] range
        action = torch.tanh(action)

        return action.squeeze(0) if batch_size == 1 else action

def load_sl_checkpoint(checkpoint_path: str):
    """Load the SL trained model from checkpoint"""
    print(f"Loading SL checkpoint from {checkpoint_path}...")

    if not os.path.exists(checkpoint_path):
        print(f"❌ Checkpoint not found: {checkpoint_path}")
        return None, None

    checkpoint = torch.load(checkpoint_path, map_location=DEVICE)

    # Extract architecture from checkpoint
    if 'architecture' in checkpoint:
        arch = checkpoint['architecture']
        print(f"✅ Found architecture in checkpoint: {arch}")
    else:
        print("⚠️  No architecture found, using default SL architecture")
        arch = {'d_model': 128, 'n_heads': 8, 'n_layers': 4, 'ffn_hidden': 512, 'max_boids': 50}

    # Create model with correct architecture
    model = TransformerPredictor(
        d_model=arch['d_model'],
        n_heads=arch['n_heads'],
        n_layers=arch['n_layers'],
        ffn_hidden=arch['ffn_hidden'],
        max_boids=arch.get('max_boids', 50),
        dropout=0.1
    ).to(DEVICE)

    # Load model state
    model.load_state_dict(checkpoint['model_state_dict'])

    # Model info
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

    print(f"✅ Loaded SL model successfully:")
    print(f"  Epoch: {checkpoint.get('epoch', 'unknown')}")
    print(f"  Best val loss: {checkpoint.get('best_val_loss', 'unknown')}")
    print(f"  Architecture: {arch['d_model']}×{arch['n_heads']}×{arch['n_layers']}×{arch['ffn_hidden']}")
    print(f"  Total parameters: {total_params:,}")
    print(f"  Trainable parameters: {trainable_params:,}")

    return model, arch

# Try to load SL checkpoint - adjust path as needed
sl_checkpoint_paths = [
    "checkpoints/best_model.pt",
    "checkpoints/model_epoch_20.pt",
    "checkpoints/model_epoch_10.pt",
    "checkpoints/model_epoch_5.pt"
]

sl_model = None
sl_architecture = None

for checkpoint_path in sl_checkpoint_paths:
    if os.path.exists(checkpoint_path):
        sl_model, sl_architecture = load_sl_checkpoint(checkpoint_path)
        if sl_model is not None:
            break

if sl_model is None:
    print("⚠️  No SL checkpoint found. Creating random initialized model for testing...")
    # Create model with default architecture for testing
    default_arch = {'d_model': 128, 'n_heads': 8, 'n_layers': 4, 'ffn_hidden': 512, 'max_boids': 50}
    sl_model = TransformerPredictor(**default_arch).to(DEVICE)
    sl_architecture = default_arch
    print(f"✅ Created model with default architecture: {default_arch}")

print(f"\n🎯 SL Model loaded and ready for RL fine-tuning!")


⚠️  No SL checkpoint found. Creating random initialized model for testing...
✅ Created model with default architecture: {'d_model': 128, 'n_heads': 8, 'n_layers': 4, 'ffn_hidden': 512, 'max_boids': 50}

🎯 SL Model loaded and ready for RL fine-tuning!


In [6]:
# Test SL Model to Ensure It Works Properly
def test_sl_model():
    """Test the loaded SL model with sample inputs"""
    print("🧪 Testing SL model with sample inputs...")

    # Create test input (same format as used in training)
    test_input = {
        'context': {'canvasWidth': 0.8, 'canvasHeight': 0.6},
        'predator': {'velX': 0.1, 'velY': -0.2},
        'boids': [
            {'relX': 0.1, 'relY': 0.3, 'velX': 0.5, 'velY': -0.1},
            {'relX': -0.2, 'relY': 0.1, 'velX': -0.3, 'velY': 0.4},
            {'relX': 0.3, 'relY': -0.1, 'velX': 0.2, 'velY': 0.3}
        ]
    }

    # Test single input
    sl_model.eval()
    with torch.no_grad():
        output = sl_model(test_input)

    print(f"✅ Single input test:")
    print(f"  Input: {len(test_input['boids'])} boids")
    print(f"  Output: [{output[0]:.4f}, {output[1]:.4f}]")
    print(f"  Output range: [{output.min():.4f}, {output.max():.4f}]")

    # Verify output is in correct range
    assert output.shape == (2,), f"Expected output shape (2,), got {output.shape}"
    assert torch.all(output >= -1) and torch.all(output <= 1), f"Output not in [-1, 1] range: {output}"

    # Test batch input
    batch_inputs = [test_input, test_input, test_input]
    with torch.no_grad():
        batch_output = sl_model(batch_inputs)

    print(f"✅ Batch input test:")
    print(f"  Batch size: {len(batch_inputs)}")
    print(f"  Output shape: {batch_output.shape}")
    print(f"  Output sample: [{batch_output[0][0]:.4f}, {batch_output[0][1]:.4f}]")

    assert batch_output.shape == (3, 2), f"Expected batch output shape (3, 2), got {batch_output.shape}"
    assert torch.all(batch_output >= -1) and torch.all(batch_output <= 1), f"Batch output not in [-1, 1] range"

    # Test with varying number of boids
    print(f"✅ Testing with different boid counts:")
    for num_boids in [1, 5, 10, 25, 50]:
        test_boids = []
        for i in range(num_boids):
            test_boids.append({
                'relX': random.uniform(-0.5, 0.5),
                'relY': random.uniform(-0.5, 0.5),
                'velX': random.uniform(-0.5, 0.5),
                'velY': random.uniform(-0.5, 0.5)
            })

        test_input_var = {
            'context': {'canvasWidth': 0.8, 'canvasHeight': 0.6},
            'predator': {'velX': 0.1, 'velY': -0.2},
            'boids': test_boids
        }

        with torch.no_grad():
            output_var = sl_model(test_input_var)

        print(f"    {num_boids} boids: [{output_var[0]:.3f}, {output_var[1]:.3f}]")
        assert output_var.shape == (2,), f"Failed for {num_boids} boids"

    print(f"🎉 All SL model tests passed!")
    return True

# Test empty boids case
def test_edge_cases():
    """Test edge cases for the model"""
    print("🔬 Testing edge cases...")

    # Test with no boids
    empty_input = {
        'context': {'canvasWidth': 0.8, 'canvasHeight': 0.6},
        'predator': {'velX': 0.1, 'velY': -0.2},
        'boids': []
    }

    sl_model.eval()
    with torch.no_grad():
        empty_output = sl_model(empty_input)

    print(f"✅ No boids test: [{empty_output[0]:.4f}, {empty_output[1]:.4f}]")
    assert empty_output.shape == (2,), "Failed no boids test"

    # Test with extreme values
    extreme_input = {
        'context': {'canvasWidth': 1.0, 'canvasHeight': 1.0},
        'predator': {'velX': 1.0, 'velY': -1.0},
        'boids': [
            {'relX': 1.0, 'relY': 1.0, 'velX': 1.0, 'velY': 1.0},
            {'relX': -1.0, 'relY': -1.0, 'velX': -1.0, 'velY': -1.0}
        ]
    }

    with torch.no_grad():
        extreme_output = sl_model(extreme_input)

    print(f"✅ Extreme values test: [{extreme_output[0]:.4f}, {extreme_output[1]:.4f}]")
    assert extreme_output.shape == (2,), "Failed extreme values test"
    assert torch.all(extreme_output >= -1) and torch.all(extreme_output <= 1), "Extreme output out of range"

    print(f"🎉 All edge case tests passed!")
    return True

# Run tests
try:
    test_sl_model()
    test_edge_cases()
    print(f"\n✅ SL Model is working correctly and ready for RL training!")

except Exception as e:
    print(f"❌ SL Model test failed: {e}")
    raise


🧪 Testing SL model with sample inputs...
✅ Single input test:
  Input: 3 boids
  Output: [0.1647, 0.0989]
  Output range: [0.0989, 0.1647]
✅ Batch input test:
  Batch size: 3
  Output shape: torch.Size([3, 2])
  Output sample: [0.1647, 0.0989]
✅ Testing with different boid counts:
    1 boids: [0.490, -0.183]
    5 boids: [0.000, 0.250]
    10 boids: [-0.189, 0.400]
    25 boids: [-0.389, 0.495]
    50 boids: [-0.447, 0.530]
🎉 All SL model tests passed!
🔬 Testing edge cases...
✅ No boids test: [0.6813, -0.3783]
✅ Extreme values test: [0.3799, 0.0528]
🎉 All edge case tests passed!

✅ SL Model is working correctly and ready for RL training!


In [7]:
# Create RL Environment Wrapper
class BoidsRLEnvironment:
    """RL Environment wrapper for boids simulation with retrospective rewards"""

    def __init__(self, config=None):
        self.config = config or ENV_CONFIG

        # Initialize simulation components
        self.state_generator = RandomStateGenerator()
        self.state_manager = StateManager()
        self.input_processor = InputProcessor()
        self.action_processor = ActionProcessor()

        # Episode tracking
        self.step_count = 0
        self.episode_buffer = []  # Store (state, action, reward) for retrospective rewards
        self.current_state = None
        self.initial_boid_count = 0

        # Episode statistics
        self.episode_stats = {
            'boids_caught': 0,
            'total_reward': 0.0,
            'catch_steps': [],  # Steps when catches occurred
        }

        print(f"✅ BoidsRLEnvironment initialized:")
        print(f"  Max episode steps: {self.config['max_episode_steps']}")
        print(f"  Reward window: {self.config['reward_window']}")
        print(f"  Base catch reward: {self.config['base_catch_reward']}")

    def reset(self):
        """Reset environment for new episode"""
        # Generate random episode configuration
        canvas_width = random.randint(*self.config['canvas_width_range'])
        canvas_height = random.randint(*self.config['canvas_height_range'])
        num_boids = random.randint(*self.config['boid_count_range'])

        # Generate initial state
        self.current_state = self.state_generator.generate_scattered_state(
            num_boids, canvas_width, canvas_height
        )

        # Reset episode tracking
        self.step_count = 0
        self.episode_buffer = []
        self.initial_boid_count = len(self.current_state['boids_states'])

        # Reset episode statistics
        self.episode_stats = {
            'boids_caught': 0,
            'total_reward': 0.0,
            'catch_steps': [],
        }

        # Get initial observation
        observation = self.input_processor.process_inputs(
            self.current_state['boids_states'],
            self.current_state['predator_state']['position'],
            self.current_state['predator_state']['velocity'],
            self.current_state['canvas_width'],
            self.current_state['canvas_height']
        )

        return observation

    def step(self, action):
        """Execute one environment step"""
        # Store current step in buffer (reward will be assigned retrospectively)
        step_data = {
            'step': self.step_count,
            'observation': self.get_current_observation(),
            'action': action.copy(),
            'reward': 0.0,  # Will be updated retrospectively
        }
        self.episode_buffer.append(step_data)

        # Convert action to simulation format
        action_forces = self.action_processor.process_action(action)
        predator_action = {
            'force_x': action_forces[0],
            'force_y': action_forces[1]
        }

        # Run simulation step
        from simulation.runtime.simulation_runtime import simulation_step
        step_result = simulation_step(
            self.current_state['boids_states'],
            self.current_state['predator_state'],
            predator_action,
            self.current_state['canvas_width'],
            self.current_state['canvas_height']
        )

        # Update state and process catches
        caught_boids = step_result['caught_boids']
        self.current_state['boids_states'] = step_result['boids_states']
        self.current_state['predator_state'] = step_result['predator_state']

        # Remove caught boids (in reverse order to maintain indices)
        for i in reversed(caught_boids):
            self.current_state['boids_states'].pop(i)

        # Assign retrospective rewards for catches
        if caught_boids:
            self.episode_stats['boids_caught'] += len(caught_boids)
            self.episode_stats['catch_steps'].append(self.step_count)
            self._assign_retrospective_rewards(self.step_count)

        # Update step count
        self.step_count += 1

        # Check termination conditions
        done = self._is_episode_done()

        # Get current reward (from buffer)
        current_reward = step_data['reward']
        self.episode_stats['total_reward'] += current_reward

        # Get next observation
        if not done:
            next_observation = self.get_current_observation()
        else:
            next_observation = None

        # Episode info
        info = {
            'boids_remaining': len(self.current_state['boids_states']),
            'boids_caught_this_step': len(caught_boids),
            'step_count': self.step_count,
            'episode_stats': self.episode_stats.copy()
        }

        return next_observation, current_reward, done, info

    def _assign_retrospective_rewards(self, catch_step):
        """Assign retrospective rewards to recent steps that led to catch"""
        reward_window = self.config['reward_window']
        base_reward = self.config['base_catch_reward']

        # Assign rewards to last N steps (more recent = higher reward)
        for i in range(reward_window):
            step_idx = catch_step - i
            if step_idx >= 0 and step_idx < len(self.episode_buffer):
                # Linear decay: more recent steps get higher reward
                reward_multiplier = (reward_window - i) / reward_window
                additional_reward = base_reward * reward_multiplier
                self.episode_buffer[step_idx]['reward'] += additional_reward

    def _is_episode_done(self):
        """Check if episode should terminate"""
        # Timeout
        if self.step_count >= self.config['max_episode_steps']:
            return True

        # All boids caught
        if len(self.current_state['boids_states']) == 0:
            return True

        return False

    def get_current_observation(self):
        """Get current observation in structured format"""
        return self.input_processor.process_inputs(
            self.current_state['boids_states'],
            self.current_state['predator_state']['position'],
            self.current_state['predator_state']['velocity'],
            self.current_state['canvas_width'],
            self.current_state['canvas_height']
        )

    def get_episode_buffer(self):
        """Get the complete episode buffer with retrospective rewards"""
        return self.episode_buffer.copy()

    def get_episode_stats(self):
        """Get episode statistics"""
        stats = self.episode_stats.copy()
        stats['efficiency'] = stats['boids_caught'] / self.initial_boid_count if self.initial_boid_count > 0 else 0
        stats['steps_per_catch'] = self.step_count / stats['boids_caught'] if stats['boids_caught'] > 0 else float('inf')
        return stats

# Create and test the RL environment
print("🏗️ Creating and testing RL environment...")

env = BoidsRLEnvironment(ENV_CONFIG)

# Test environment reset
print("\n🧪 Testing environment reset...")
obs = env.reset()
print(f"✅ Reset successful:")
print(f"  Initial boids: {len(obs['boids'])}")
print(f"  Canvas size: {obs['context']['canvasWidth']:.3f} x {obs['context']['canvasHeight']:.3f}")
print(f"  Predator velocity: [{obs['predator']['velX']:.3f}, {obs['predator']['velY']:.3f}]")

# Test environment step with random actions
print("\n🧪 Testing environment steps...")
for i in range(5):
    random_action = [random.uniform(-1, 1), random.uniform(-1, 1)]
    next_obs, reward, done, info = env.step(random_action)

    print(f"  Step {i+1}: action=[{random_action[0]:.3f}, {random_action[1]:.3f}], reward={reward:.3f}, boids={info['boids_remaining']}, done={done}")

    if done:
        print(f"  Episode terminated at step {i+1}")
        break

print(f"\n✅ Environment test completed!")
print(f"  Final episode stats: {env.get_episode_stats()}")
print(f"🎯 RL Environment is ready for training!")

🏗️ Creating and testing RL environment...
✅ BoidsRLEnvironment initialized:
  Max episode steps: 3000
  Reward window: 50
  Base catch reward: 1.0

🧪 Testing environment reset...
✅ Reset successful:
  Initial boids: 1
  Canvas size: 1.425 x 0.853
  Predator velocity: [0.424, -0.090]

🧪 Testing environment steps...
  Step 1: action=[-0.187, -0.874], reward=0.000, boids=1, done=False
  Step 2: action=[-0.036, 0.377], reward=0.000, boids=1, done=False
  Step 3: action=[0.193, 0.424], reward=0.000, boids=1, done=False
  Step 4: action=[-0.879, -0.480], reward=0.000, boids=1, done=False
  Step 5: action=[0.886, 0.416], reward=0.000, boids=1, done=False

✅ Environment test completed!
  Final episode stats: {'boids_caught': 0, 'total_reward': 0.0, 'catch_steps': [], 'efficiency': 0.0, 'steps_per_catch': inf}
🎯 RL Environment is ready for training!


In [8]:
# Define Critic Network and PPO Components
class TransformerCritic(nn.Module):
    """Transformer-based value function critic"""

    def __init__(self, d_model=64, n_heads=4, n_layers=2, ffn_hidden=256, max_boids=50, dropout=0.1):
        super().__init__()
        self.d_model = d_model
        self.n_heads = n_heads
        self.n_layers = n_layers
        self.ffn_hidden = ffn_hidden
        self.max_boids = max_boids

        # CLS token embedding
        self.cls_embedding = nn.Parameter(torch.randn(d_model))

        # Type embeddings
        self.type_embeddings = nn.ParameterDict({
            'cls': nn.Parameter(torch.randn(d_model)),
            'ctx': nn.Parameter(torch.randn(d_model)),
            'predator': nn.Parameter(torch.randn(d_model)),
            'boid': nn.Parameter(torch.randn(d_model))
        })

        # Input projections (same as actor but smaller dimension)
        self.ctx_projection = nn.Linear(2, d_model)
        self.predator_projection = nn.Linear(4, d_model)
        self.boid_projection = nn.Linear(4, d_model)

        # Transformer layers (lighter than actor)
        self.transformer_layers = nn.ModuleList([
            TransformerLayer(d_model, n_heads, ffn_hidden, dropout)
            for _ in range(n_layers)
        ])

        # Value head
        self.value_head = nn.Linear(d_model, 1)

    def forward(self, structured_inputs):
        batch_size = len(structured_inputs) if isinstance(structured_inputs, list) else 1

        # Handle single sample vs batch
        if isinstance(structured_inputs, dict):
            structured_inputs = [structured_inputs]
            batch_size = 1

        # Build token sequences (same as actor)
        sequences = []
        masks = []

        for sample in structured_inputs:
            tokens = []

            # CLS token
            cls_token = self.cls_embedding + self.type_embeddings['cls']
            tokens.append(cls_token)

            # Context token
            ctx_input = torch.tensor([sample['context']['canvasWidth'], sample['context']['canvasHeight']],
                                   dtype=torch.float32, device=self.cls_embedding.device)
            ctx_token = self.ctx_projection(ctx_input) + self.type_embeddings['ctx']
            tokens.append(ctx_token)

            # Predator token
            predator_input = torch.tensor([sample['predator']['velX'], sample['predator']['velY'], 0.0, 0.0],
                                        dtype=torch.float32, device=self.cls_embedding.device)
            predator_token = self.predator_projection(predator_input) + self.type_embeddings['predator']
            tokens.append(predator_token)

            # Boid tokens
            sample_mask = [False, False, False]

            for boid in sample['boids']:
                boid_input = torch.tensor([boid['relX'], boid['relY'], boid['velX'], boid['velY']],
                                        dtype=torch.float32, device=self.cls_embedding.device)
                boid_token = self.boid_projection(boid_input) + self.type_embeddings['boid']
                tokens.append(boid_token)
                sample_mask.append(False)

            # Pad to max_boids + 3
            while len(tokens) < self.max_boids + 3:
                padding_token = torch.zeros(self.d_model, device=self.cls_embedding.device)
                tokens.append(padding_token)
                sample_mask.append(True)

            sequences.append(torch.stack(tokens))
            masks.append(sample_mask)

        # Stack sequences
        x = torch.stack(sequences)  # [batch_size, seq_len, d_model]

        # Create padding mask
        padding_mask = torch.tensor(masks, dtype=torch.bool, device=x.device)

        # Pass through transformer layers
        for layer in self.transformer_layers:
            x = layer(x, padding_mask)

        # Extract CLS token and compute value
        cls_output = x[:, 0]  # [batch_size, d_model]
        value = self.value_head(cls_output)  # [batch_size, 1]

        return value.squeeze(-1) if batch_size == 1 else value.squeeze(-1)

class PPOBuffer:
    """Buffer for storing PPO rollout data"""

    def __init__(self, size, gamma=0.99, lam=0.95):
        self.size = size
        self.gamma = gamma
        self.lam = lam

        # Storage
        self.observations = []
        self.actions = []
        self.rewards = []
        self.values = []
        self.log_probs = []
        self.dones = []

        # Computed values
        self.advantages = []
        self.returns = []

        self.ptr = 0
        self.full = False

    def store(self, obs, action, reward, value, log_prob, done):
        """Store transition"""
        if len(self.observations) < self.size:
            self.observations.append(obs)
            self.actions.append(action)
            self.rewards.append(reward)
            self.values.append(value)
            self.log_probs.append(log_prob)
            self.dones.append(done)
        else:
            self.observations[self.ptr] = obs
            self.actions[self.ptr] = action
            self.rewards[self.ptr] = reward
            self.values[self.ptr] = value
            self.log_probs[self.ptr] = log_prob
            self.dones[self.ptr] = done

        self.ptr = (self.ptr + 1) % self.size
        if self.ptr == 0:
            self.full = True

    def finish_path(self, last_value=0.0):
        """Compute advantages and returns for the stored path"""
        path_len = len(self.rewards)
        if path_len == 0:
            return

        # Convert to numpy for easier computation
        rewards = np.array(self.rewards[-path_len:])
        values = np.array(self.values[-path_len:] + [last_value])
        dones = np.array(self.dones[-path_len:])

        # Compute GAE advantages
        advantages = np.zeros_like(rewards)
        lastgaelam = 0

        for t in reversed(range(path_len)):
            if t == path_len - 1:
                nextnonterminal = 1.0 - dones[t]
                nextvalues = last_value
            else:
                nextnonterminal = 1.0 - dones[t]
                nextvalues = values[t + 1]

            delta = rewards[t] + self.gamma * nextvalues * nextnonterminal - values[t]
            advantages[t] = lastgaelam = delta + self.gamma * self.lam * nextnonterminal * lastgaelam

        # Compute returns
        returns = advantages + values[:-1]

        # Store computed values
        if len(self.advantages) < len(advantages):
            self.advantages.extend(advantages.tolist())
            self.returns.extend(returns.tolist())
        else:
            start_idx = len(self.advantages) - len(advantages)
            self.advantages[start_idx:] = advantages.tolist()
            self.returns[start_idx:] = returns.tolist()

    def get(self):
        """Get all stored data"""
        assert len(self.advantages) == len(self.observations), "Must call finish_path before get"

        data = {
            'observations': self.observations.copy(),
            'actions': np.array(self.actions),
            'returns': np.array(self.returns),
            'advantages': np.array(self.advantages),
            'log_probs': np.array(self.log_probs),
        }

        # Normalize advantages
        data['advantages'] = (data['advantages'] - data['advantages'].mean()) / (data['advantages'].std() + 1e-8)

        return data

    def clear(self):
        """Clear the buffer"""
        self.observations = []
        self.actions = []
        self.rewards = []
        self.values = []
        self.log_probs = []
        self.dones = []
        self.advantages = []
        self.returns = []
        self.ptr = 0
        self.full = False

# Create actor and critic networks
print("🧠 Creating actor and critic networks...")

# Actor: Use the loaded SL model
actor = sl_model
actor.train()  # Set to training mode for RL fine-tuning

# Critic: Create smaller transformer for value estimation
critic_config = {
    'd_model': 64,      # Smaller than actor
    'n_heads': 4,       # Fewer heads
    'n_layers': 2,      # Fewer layers
    'ffn_hidden': 256,  # Smaller FFN
    'max_boids': sl_architecture['max_boids'],
    'dropout': 0.1
}

critic = TransformerCritic(**critic_config).to(DEVICE)

# Count parameters
actor_params = sum(p.numel() for p in actor.parameters())
critic_params = sum(p.numel() for p in critic.parameters())

print(f"✅ Networks created:")
print(f"  Actor (SL model): {actor_params:,} parameters")
print(f"  Critic: {critic_params:,} parameters")
print(f"  Total: {actor_params + critic_params:,} parameters")

# Test networks with sample input
print("\\n🧪 Testing actor and critic...")
test_obs = env.reset()

# Test actor
actor.eval()
with torch.no_grad():
    actor_output = actor(test_obs)
print(f"✅ Actor test: output shape {actor_output.shape}, range [{actor_output.min():.3f}, {actor_output.max():.3f}]")

# Test critic
critic.eval()
with torch.no_grad():
    critic_output = critic(test_obs)
print(f"✅ Critic test: output shape {critic_output.shape}, value {critic_output.item():.3f}")

print(f"\\n🎯 Actor and Critic networks ready for PPO training!")


🧠 Creating actor and critic networks...
✅ Networks created:
  Actor (SL model): 1,059,842 parameters
  Critic: 134,465 parameters
  Total: 1,194,307 parameters
\n🧪 Testing actor and critic...
✅ Actor test: output shape torch.Size([2]), range [-0.346, 0.500]
✅ Critic test: output shape torch.Size([1]), value -0.080
\n🎯 Actor and Critic networks ready for PPO training!


In [9]:
# PPO Training Algorithm
class PPOTrainer:
    """PPO trainer for the boids RL environment"""

    def __init__(self, actor, critic, env, config):
        self.actor = actor
        self.critic = critic
        self.env = env
        self.config = config

        # Optimizers
        self.actor_optimizer = optim.AdamW(actor.parameters(), lr=config['learning_rate'])
        self.critic_optimizer = optim.AdamW(critic.parameters(), lr=config['learning_rate'])

        # PPO buffer
        self.buffer = PPOBuffer(
            size=config['rollout_length'],
            gamma=config['gamma'],
            lam=config['gae_lambda']
        )

        # Training metrics
        self.training_metrics = defaultdict(list)

        print(f"✅ PPO Trainer initialized:")
        print(f"  Learning rate: {config['learning_rate']}")
        print(f"  Rollout length: {config['rollout_length']}")
        print(f"  Clip epsilon: {config['clip_epsilon']}")

    def get_action_and_value(self, obs, deterministic=False):
        """Get action and value from networks"""
        self.actor.eval()
        self.critic.eval()

        with torch.no_grad():
            # Get action from actor (deterministic output)
            action_mean = self.actor(obs)

            # Add exploration noise for stochastic policy
            if not deterministic:
                # Use fixed std for simplicity (could be learned)
                action_std = 0.1
                action = action_mean + torch.randn_like(action_mean) * action_std
                # Clip to [-1, 1] range
                action = torch.clamp(action, -1, 1)

                # Compute log probability (assuming Gaussian)
                log_prob = -0.5 * (((action - action_mean) / action_std) ** 2 + 2 * np.log(action_std) + np.log(2 * np.pi))
                log_prob = log_prob.sum()  # Sum across action dimensions
            else:
                action = torch.clamp(action_mean, -1, 1)
                log_prob = torch.tensor(0.0)

            # Get value from critic
            value = self.critic(obs)

        # Convert everything to CPU and extract values
        return action.cpu().numpy(), value.cpu().item(), log_prob.cpu().item()

    def collect_rollout(self):
        """Collect a rollout of experiences"""
        obs = self.env.reset()

        for step in range(self.config['rollout_length']):
            # Get action and value
            action, value, log_prob = self.get_action_and_value(obs)

            # Take step in environment
            next_obs, reward, done, info = self.env.step(action)

            # Store in buffer (all values are now CPU scalars/arrays)
            self.buffer.store(obs, action, reward, value, log_prob, done)

            obs = next_obs

            if done:
                obs = self.env.reset()

        # Always finish the path at the end of rollout
        if len(self.buffer.observations) > 0:
            # Get final value for advantage computation
            _, final_value, _ = self.get_action_and_value(obs)

            # If last episode didn't complete, use the final value; otherwise use 0
            last_done = self.buffer.dones[-1] if self.buffer.dones else False
            last_value = 0.0 if last_done else final_value

            self.buffer.finish_path(last_value=last_value)

    def update_networks(self):
        """Update actor and critic networks using PPO"""
        # Get rollout data
        data = self.buffer.get()

        # Convert to tensors
        observations = data['observations']
        actions = torch.tensor(data['actions'], dtype=torch.float32, device=DEVICE)
        returns = torch.tensor(data['returns'], dtype=torch.float32, device=DEVICE)
        advantages = torch.tensor(data['advantages'], dtype=torch.float32, device=DEVICE)
        old_log_probs = torch.tensor(data['log_probs'], dtype=torch.float32, device=DEVICE)

        # Training metrics for this update
        policy_losses = []
        value_losses = []
        kl_divergences = []
        clip_fractions = []

        # PPO update epochs
        for epoch in range(self.config['epochs_per_update']):
            # Shuffle data
            batch_size = self.config['batch_size']
            indices = torch.randperm(len(observations))

            # Mini-batch updates
            for start in range(0, len(observations), batch_size):
                end = start + batch_size
                mb_indices = indices[start:end]

                mb_obs = [observations[i] for i in mb_indices]
                mb_actions = actions[mb_indices]
                mb_returns = returns[mb_indices]
                mb_advantages = advantages[mb_indices]
                mb_old_log_probs = old_log_probs[mb_indices]

                # Forward pass through networks
                self.actor.train()
                self.critic.train()

                # Actor forward pass
                action_means = self.actor(mb_obs)

                # Compute log probabilities (assuming fixed std)
                action_std = 0.1
                log_probs = -0.5 * (((mb_actions - action_means) / action_std) ** 2 +
                                   2 * np.log(action_std) + np.log(2 * np.pi))
                log_probs = log_probs.sum(dim=-1)  # Sum across action dimensions

                # Critic forward pass
                values = self.critic(mb_obs)

                # PPO policy loss
                ratio = torch.exp(log_probs - mb_old_log_probs)
                surr1 = ratio * mb_advantages
                surr2 = torch.clamp(ratio, 1 - self.config['clip_epsilon'],
                                  1 + self.config['clip_epsilon']) * mb_advantages
                policy_loss = -torch.min(surr1, surr2).mean()

                # Value loss
                value_loss = F.mse_loss(values, mb_returns)

                # Entropy bonus (simple approximation)
                entropy = 0.5 * (1 + np.log(2 * np.pi * action_std**2))
                entropy_loss = -self.config['entropy_coef'] * entropy

                # Total actor loss
                actor_loss = policy_loss + entropy_loss

                # Update actor
                self.actor_optimizer.zero_grad()
                actor_loss.backward()
                torch.nn.utils.clip_grad_norm_(self.actor.parameters(), 0.5)
                self.actor_optimizer.step()

                # Update critic
                self.critic_optimizer.zero_grad()
                critic_loss = self.config['value_loss_coef'] * value_loss
                critic_loss.backward()
                torch.nn.utils.clip_grad_norm_(self.critic.parameters(), 0.5)
                self.critic_optimizer.step()

                # Collect metrics
                with torch.no_grad():
                    kl_div = (mb_old_log_probs - log_probs).mean()
                    clip_frac = ((ratio - 1).abs() > self.config['clip_epsilon']).float().mean()

                    policy_losses.append(policy_loss.item())
                    value_losses.append(value_loss.item())
                    kl_divergences.append(kl_div.item())
                    clip_fractions.append(clip_frac.item())

        # Store training metrics
        self.training_metrics['policy_loss'].append(np.mean(policy_losses))
        self.training_metrics['value_loss'].append(np.mean(value_losses))
        self.training_metrics['kl_divergence'].append(np.mean(kl_divergences))
        self.training_metrics['clip_fraction'].append(np.mean(clip_fractions))

        # Clear buffer
        self.buffer.clear()

    def train_step(self):
        """Single training step: collect rollout + update networks"""
        # Collect rollout
        self.collect_rollout()

        # Update networks
        self.update_networks()

        return self.training_metrics

# Create PPO trainer
print("🎯 Creating PPO trainer...")
trainer = PPOTrainer(actor, critic, env, PPO_CONFIG)

# Test a single training step
print("\n🧪 Testing a single training step...")
try:
    start_time = time.time()
    metrics = trainer.train_step()
    step_time = time.time() - start_time

    print(f"✅ Training step completed in {step_time:.2f}s")
    print(f"  Policy loss: {metrics['policy_loss'][-1]:.4f}")
    print(f"  Value loss: {metrics['value_loss'][-1]:.4f}")
    print(f"  KL divergence: {metrics['kl_divergence'][-1]:.6f}")
    print(f"  Clip fraction: {metrics['clip_fraction'][-1]:.3f}")

except Exception as e:
    print(f"❌ Training step failed: {e}")
    raise

print(f"\n🚀 PPO trainer is ready for full training!")

🎯 Creating PPO trainer...
✅ PPO Trainer initialized:
  Learning rate: 3e-05
  Rollout length: 2048
  Clip epsilon: 0.1

🧪 Testing a single training step...
✅ Training step completed in 148.76s
  Policy loss: 0.0492
  Value loss: 0.0044
  KL divergence: 0.343534
  Clip fraction: 0.827

🚀 PPO trainer is ready for full training!


In [ ]:
# Main Training Loop with Metrics and Monitoring
class TrainingMonitor:
    """Monitor training progress with comprehensive metrics"""

    def __init__(self):
        self.performance_metrics = defaultdict(list)
        self.learning_metrics = defaultdict(list)
        self.behavioral_metrics = defaultdict(list)

        self.episode_count = 0
        self.step_count = 0

    def update_episode_metrics(self, episode_stats, episode_reward):
        """Update metrics after episode completion"""
        self.episode_count += 1

        # Performance metrics
        self.performance_metrics['boids_caught_per_episode'].append(episode_stats['boids_caught'])
        self.performance_metrics['catch_efficiency'].append(episode_stats['efficiency'])
        self.performance_metrics['episode_reward'].append(episode_reward)
        self.performance_metrics['episode_length'].append(episode_stats.get('step_count', 0))

        # Success rate (rolling average)
        success = 1.0 if episode_stats['efficiency'] >= 0.8 else 0.0
        self.performance_metrics['success_rate'].append(success)

        # Catch timing metrics
        if episode_stats['boids_caught'] > 0:
            self.performance_metrics['catches_per_episode'].append(episode_stats['boids_caught'])
            self.performance_metrics['steps_per_catch'].append(episode_stats['steps_per_catch'])

    def update_training_metrics(self, training_metrics):
        """Update learning metrics from PPO training"""
        for key, values in training_metrics.items():
            if values:  # Only add if there are values
                self.learning_metrics[key].append(values[-1])

    def get_recent_performance(self, window=100):
        """Get recent performance statistics"""
        if len(self.performance_metrics['episode_reward']) < window:
            window = len(self.performance_metrics['episode_reward'])

        if window == 0:
            return {}

        recent_rewards = self.performance_metrics['episode_reward'][-window:]
        recent_catches = self.performance_metrics['boids_caught_per_episode'][-window:]
        recent_efficiency = self.performance_metrics['catch_efficiency'][-window:]
        recent_success = self.performance_metrics['success_rate'][-window:]

        return {
            'avg_reward': np.mean(recent_rewards),
            'avg_catches': np.mean(recent_catches),
            'avg_efficiency': np.mean(recent_efficiency),
            'success_rate': np.mean(recent_success),
            'episodes': window
        }

    def plot_training_progress(self):
        """Plot training progress"""
        fig, axes = plt.subplots(2, 3, figsize=(15, 8))

        # Performance metrics
        if self.performance_metrics['episode_reward']:
            axes[0, 0].plot(self.performance_metrics['episode_reward'])
            axes[0, 0].set_title('Episode Reward')
            axes[0, 0].set_xlabel('Episode')
            axes[0, 0].grid(True)

        if self.performance_metrics['boids_caught_per_episode']:
            axes[0, 1].plot(self.performance_metrics['boids_caught_per_episode'])
            axes[0, 1].set_title('Boids Caught per Episode')
            axes[0, 1].set_xlabel('Episode')
            axes[0, 1].grid(True)

        if self.performance_metrics['catch_efficiency']:
            axes[0, 2].plot(self.performance_metrics['catch_efficiency'])
            axes[0, 2].set_title('Catch Efficiency')
            axes[0, 2].set_xlabel('Episode')
            axes[0, 2].set_ylim([0, 1])
            axes[0, 2].grid(True)

        # Learning metrics
        if self.learning_metrics['policy_loss']:
            axes[1, 0].plot(self.learning_metrics['policy_loss'])
            axes[1, 0].set_title('Policy Loss')
            axes[1, 0].set_xlabel('Update')
            axes[1, 0].grid(True)

        if self.learning_metrics['value_loss']:
            axes[1, 1].plot(self.learning_metrics['value_loss'])
            axes[1, 1].set_title('Value Loss')
            axes[1, 1].set_xlabel('Update')
            axes[1, 1].grid(True)

        if self.learning_metrics['kl_divergence']:
            axes[1, 2].plot(self.learning_metrics['kl_divergence'])
            axes[1, 2].set_title('KL Divergence')
            axes[1, 2].set_xlabel('Update')
            axes[1, 2].grid(True)

        plt.tight_layout()
        plt.show()

def save_checkpoint(actor, critic, trainer, monitor, update, is_best=False):
    """Save training checkpoint"""
    os.makedirs("checkpoints_rl", exist_ok=True)

    checkpoint = {
        'update': update,
        'actor_state_dict': actor.state_dict(),
        'critic_state_dict': critic.state_dict(),
        'actor_optimizer_state_dict': trainer.actor_optimizer.state_dict(),
        'critic_optimizer_state_dict': trainer.critic_optimizer.state_dict(),
        'training_metrics': dict(trainer.training_metrics),
        'performance_metrics': dict(monitor.performance_metrics),
        'architecture': sl_architecture,
        'config': PPO_CONFIG,
        'timestamp': datetime.now().isoformat()
    }

    # Save regular checkpoint
    checkpoint_path = f"checkpoints_rl/rl_model_update_{update}.pt"
    torch.save(checkpoint, checkpoint_path)
    print(f"✓ Saved checkpoint: {checkpoint_path}")

    # Save best model
    if is_best:
        best_path = "checkpoints_rl/best_rl_model.pt"
        torch.save(checkpoint, best_path)
        print(f"✓ Saved best model: {best_path}")

    return checkpoint_path

def train_ppo(trainer, monitor, num_updates=100, eval_interval=10, save_interval=25):
    """Main PPO training loop"""
    print(f"🚀 Starting PPO training for {num_updates} updates...")
    print(f"  Evaluation interval: {eval_interval}")
    print(f"  Save interval: {save_interval}")

    best_avg_reward = -np.inf
    start_time = time.time()

    for update in range(1, num_updates + 1):
        update_start_time = time.time()

        # Collect rollout and train
        metrics = trainer.train_step()

        # Update learning metrics
        monitor.update_training_metrics(metrics)

        # Evaluation episodes
        if update % eval_interval == 0:
            print(f"\\n--- Update {update}/{num_updates} Evaluation ---")

            eval_rewards = []
            eval_stats = []

            # Run evaluation episodes
            for eval_ep in range(5):  # 5 evaluation episodes
                obs = env.reset()
                episode_reward = 0
                done = False

                while not done:
                    action, _, _ = trainer.get_action_and_value(obs, deterministic=True)
                    obs, reward, done, info = env.step(action)
                    episode_reward += reward

                eval_rewards.append(episode_reward)
                eval_stats.append(env.get_episode_stats())

                # Update episode metrics
                monitor.update_episode_metrics(env.get_episode_stats(), episode_reward)

            # Evaluation summary
            avg_reward = np.mean(eval_rewards)
            avg_catches = np.mean([stats['boids_caught'] for stats in eval_stats])
            avg_efficiency = np.mean([stats['efficiency'] for stats in eval_stats])

            print(f"  Avg reward: {avg_reward:.3f}")
            print(f"  Avg catches: {avg_catches:.1f}")
            print(f"  Avg efficiency: {avg_efficiency:.3f}")

            # Recent performance
            recent_perf = monitor.get_recent_performance(window=50)
            if recent_perf:
                print(f"  Recent performance (50 episodes):")
                print(f"    Avg reward: {recent_perf['avg_reward']:.3f}")
                print(f"    Success rate: {recent_perf['success_rate']:.3f}")

            # Check if best model
            is_best = avg_reward > best_avg_reward
            if is_best:
                best_avg_reward = avg_reward
                print(f"  🎯 New best average reward: {best_avg_reward:.3f}")

            # Learning metrics
            if metrics['policy_loss']:
                print(f"  Policy loss: {metrics['policy_loss'][-1]:.4f}")
                print(f"  Value loss: {metrics['value_loss'][-1]:.4f}")
                print(f"  KL divergence: {metrics['kl_divergence'][-1]:.6f}")

        # Save checkpoint
        if update % save_interval == 0 or update == num_updates:
            save_checkpoint(actor, critic, trainer, monitor, update,
                          is_best=(update % eval_interval == 0 and avg_reward > best_avg_reward))

        # Progress update
        update_time = time.time() - update_start_time
        elapsed_time = time.time() - start_time
        remaining_time = (elapsed_time / update) * (num_updates - update)

        if update % eval_interval == 0:
            print(f"  Update time: {update_time:.1f}s, Elapsed: {elapsed_time/60:.1f}min, ETA: {remaining_time/60:.1f}min")

    print(f"\\n🎉 Training completed!")
    print(f"  Total time: {(time.time() - start_time)/60:.1f} minutes")
    print(f"  Best average reward: {best_avg_reward:.3f}")

    return monitor

# Initialize training monitor
monitor = TrainingMonitor()

# Run training (adjust num_updates as needed)
print("🏁 Starting RL training...")
print("Note: Adjust num_updates based on available time and compute")

# Small test run first
print("\\n🧪 Running a small test training (5 updates)...")
try:
    test_monitor = train_ppo(trainer, monitor, num_updates=100, eval_interval=5, save_interval=5)
    print("✅ Test training completed successfully!")

    # Plot initial results
    print("\\n📊 Plotting test training progress...")
    test_monitor.plot_training_progress()

except Exception as e:
    print(f"❌ Test training failed: {e}")
    raise

print(f"\\n🎯 RL Training system is ready!")
print(f"To run full training, call:")
print(f"  final_monitor = train_ppo(trainer, monitor, num_updates=100, eval_interval=10, save_interval=25)")
print(f"\\nThe trained model can then be exported back to JavaScript using export_to_js.py")


🏁 Starting RL training...
Note: Adjust num_updates based on available time and compute
\n🧪 Running a small test training (5 updates)...
🚀 Starting PPO training for 100 updates...
  Evaluation interval: 5
  Save interval: 5
\n--- Update 5/100 Evaluation ---
  Avg reward: 1.000
  Avg catches: 1.0
  Avg efficiency: 0.081
  Recent performance (50 episodes):
    Avg reward: 1.000
    Success rate: 0.000
  🎯 New best average reward: 1.000
  Policy loss: 0.1175
  Value loss: 0.0002
  KL divergence: 0.158057
✓ Saved checkpoint: checkpoints_rl/rl_model_update_5.pt
  Update time: 219.4s, Elapsed: 9.9min, ETA: 187.8min
\n--- Update 10/100 Evaluation ---
  Avg reward: 2.800
  Avg catches: 2.8
  Avg efficiency: 0.080
  Recent performance (50 episodes):
    Avg reward: 1.900
    Success rate: 0.000
  🎯 New best average reward: 2.800
  Policy loss: 0.0229
  Value loss: 0.0042
  KL divergence: 0.056886
✓ Saved checkpoint: checkpoints_rl/rl_model_update_10.pt
  Update time: 354.4s, Elapsed: 21.3min, ET